In [1]:
import rasterio
import numpy as np
from netCDF4 import Dataset
from affine import Affine
import os
from datetime import datetime
import CRS



In [2]:
# Function to read GeoTIFF metadata
def read_geotiff_metadata(tif_file):
    with rasterio.open(tif_file) as dataset:
        meta = dataset.meta
        transform = dataset.transform
        crs = dataset.crs
        data = dataset.read()  # Reads all bands
        band1 = dataset.read(1)  # Reads the first band
        nodata = dataset.nodata
        width = dataset.width
        height = dataset.height
        count = dataset.count
        bounds = dataset.bounds

    return {
        'meta': meta,
        'transform': transform,
        'crs': crs,
        'data': data,
        'band1': band1,
        'nodata': nodata,
        'width': width,
        'height': height,
        'count': count,
        'bounds': bounds,
    }

geotiff_info = read_geotiff_metadata('./ftp_download/AWD_0-40cm_2024-06-14.tif')
print(geotiff_info)


{'meta': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 259, 'height': 191, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.1, 0.0, 4.650000000000006,
       0.0, -0.1, 55.050000000000004)}, 'transform': Affine(0.1, 0.0, 4.650000000000006,
       0.0, -0.1, 55.050000000000004), 'crs': CRS.from_epsg(4326), 'data': array([[[-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          1.17666445e+01,  8.86395264e+00,  6.19990253e+00],
        [-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          4.17536259e+00,  2.97724628e+00, -2.62212873e+00],
        [-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          2.37905431e+00, -4.16109848e+00, -3.93445301e+00],
        ...,
        [-3.40282347e+38, -2.22427845e+00, -5.01790047e+00, ...,
         -1.19066644e+00, -1.19066644e+00, -1.19066644e+00],
        [-3.40282347e+38, -1.79855525e+00, -2.24067092e+00, ...,
         -3.40282347e+38, -3.40282347e+38, -3.402823

In [10]:
def create_netcdf(path_dict, output_dir='./netcdfs_8'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for name, data in path_dict.items():
        netcdf_file_path = os.path.join(output_dir, f'{name}.nc')

        width = data['tif_meta']['width']
        height = data['tif_meta']['height']
        dates = data['dates']
        num_dates = len(dates)



        with Dataset(netcdf_file_path, 'w', format='NETCDF4') as dst:
            dst.createDimension('lon', width)
            dst.createDimension('lat', height)
            dst.createDimension('time', num_dates)

            x_var = dst.createVariable('lon', 'f4', ('lon',))
            y_var = dst.createVariable('lat', 'f4', ('lat',))
            time_var = dst.createVariable('time', 'f4', ('time',))
            utci_var = dst.createVariable('UTCI', 'f4', ('time', 'lat', 'lon'), fill_value=data['tif_meta']['nodata'])

            x_var[:] = lons
            y_var[:] = lats
            time_var[:] = np.arange(num_dates)
            utci_var[:] = np.array(data['stack'])

            # Set CRS attribute
            crs_wkt = data['tif_meta']['crs'].to_wkt()
            dst.setncattr('crs', crs_wkt)
            
            # Set affine transformation attribute correctly
            transform_params = (transform.a, transform.b, transform.c, transform.d, transform.e, transform.f)
            dst.setncattr('transform', ', '.join(map(str, transform_params)))

            dst.setncattr('nodata', data['tif_meta']['nodata'])

            if data['upper_limit'] is not None:
                dst.setncattr('upper_limit', data['upper_limit'])
                dst.setncattr('lower_limit', data['lower_limit'])

            dst.setncattr('dates', dates)
            dst.setncattr('history', 'Created ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))




In [6]:
def load_tif_stacks(input_dir):
    tif_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]
    tif_files.sort()
    sublist = []
    counter = 0
    input_name = ''
    path_dict = {}
    
    for tif_name in tif_files:
        name_parts = tif_name.split('.tif')[0].split('_')
        iso_date = name_parts[-1]
        name_parts.pop(-1)
        if len(name_parts) > 1:
            name = '_'.join(name_parts)
        else:
            name = name_parts[0]

        if input_name != name:
            input_name = name
            path_dict[name] = {
                'dates': [],
                'standard_name': name_parts[0],
                'tif_meta': None,
                'upper_limit': None,
                'lower_limit': None,
                'stack': [],
                'paths': []
            }

            if len(name_parts) > 1:
                try:
                    path_dict[name]['upper_limit'] = int(name_parts[-1].split('cm')[0].split('-')[0])
                    path_dict[name]['lower_limit'] = int(name_parts[-1].split('cm')[0].split('-')[1])
                except:
                    continue


                    

        with rasterio.open(os.path.join(input_dir, tif_name)) as src:
            path_dict[name]['tif_meta'] = src.meta
            path_dict[name]['stack'].append(src.read(1))

        path_dict[name]['dates'].append(iso_date)
        path_dict[name]['paths'].append(os.path.join(input_dir, tif_name))


    return path_dict

In [7]:
path_dict = load_tif_stacks('./ftp_download')

In [11]:
create_netcdf(path_dict)

In [126]:
path_dict

{'AWD_0-100cm': {'dates': ['2024-06-14',
   '2024-06-15',
   '2024-06-16',
   '2024-06-17',
   '2024-06-18',
   '2024-06-19',
   '2024-06-20',
   '2024-06-21',
   '2024-06-22',
   '2024-06-23'],
  'standard_name': 'AWD',
  'tif_meta': {'driver': 'GTiff',
   'dtype': 'float32',
   'nodata': -3.4028234663852886e+38,
   'width': 259,
   'height': 191,
   'count': 1,
   'crs': CRS.from_epsg(4326),
   'transform': Affine(0.1, 0.0, 4.650000000000006,
          0.0, -0.1, 55.050000000000004)},
  'upper_limit': 0,
  'lower_limit': 100,
  'stack': [array([[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
            4.7473812e+00,  9.0766674e-01, -2.5179274e+00],
          [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
           -4.8436737e+00, -5.6127849e+00, -1.1220232e+01],
          [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
           -6.5563340e+00, -1.0986254e+01, -1.0359383e+01],
          ...,
          [-3.4028235e+38, -6.5413370e+00, -9.0777950e+00, ...,
      

In [136]:
path = path_dict['UTCI']['paths'][0]

with Dataset('./netcdfs/test21.nc', 'w', format='NETCDF4') as dst:
    time = dst.createDimension('time', None) # unlimited dimension, but actually len(path_dict['UTCI']['dates'])
    lat = dst.createDimension('lat', path_dict['UTCI']['tif_meta']['height'])
    lon = dst.createDimension('lon', path_dict['UTCI']['tif_meta']['width'])
    
    times = dst.createVariable('time', 'i2', ('time',))
    latitudes = dst.createVariable('lat', 'f4', ('lat',))
    longitudes = dst.createVariable('lon', 'f4', ('lon',))
    
    time_length = len(path_dict['UTCI']['dates'])
    
    
    variable = dst.createVariable('utci', 'f4', ('time', 'lat', 'lon'), fill_value=path_dict['UTCI']['tif_meta']['nodata'])
    
    times[:] = np.arange(time_length)
    times.units = f'days since {path_dict['UTCI']['dates'][0]} 00:00:00'
    times.calendar = 'gregorian'

    start_lon = path_dict['UTCI']['tif_meta']['transform'][2]
    lon_step = path_dict['UTCI']['tif_meta']['transform'][0]
    start_lon = start_lon + lon_step/2
    lon_len = path_dict['UTCI']['tif_meta']['width']

    start_lat = path_dict['UTCI']['tif_meta']['transform'][5]
    lat_step  = path_dict['UTCI']['tif_meta']['transform'][4]
    start_lat = start_lat + lat_step/2
    lat_len = path_dict['UTCI']['tif_meta']['height']

    lons = np.array([start_lon + i * lon_step for i in range(lon_len)])
    lats = np.array([start_lat + i * lat_step for i in range(lat_len)])

    latitudes[:] = lats[:]
    longitudes[:] = lons[:]
    
    latitudes.units = 'degree_north'
    latitudes.long_name = 'latitude'
    latitudes.standard_name = 'latitude'
    latitudes.axis = 'Y'

    longitudes.units = 'degree_east'
    longitudes.long_name = 'longitude'
    longitudes.standard_name = 'longitude'
    longitudes.axis = 'X'  

    variable[:] = np.array(path_dict['UTCI']['stack'])

    variable.units = 'K'
    variable.long_name = 'Universal Thermal Climate Index'
    variable.setncattr('minimum_value', np.nanmin(variable[:]))
    variable.setncattr('maximum_value', np.nanmax(variable[:]))





    # dst['lon'][:] = lats[:]
    # dst['lat'][:] = lons[:]
    # dst['time'][:] = np.arange(len(path_dict['UTCI']['dates']))

    # Set CRS attribute
    crs_wkt = path_dict['UTCI']['tif_meta']['crs'].to_wkt()

    # dst['utci'][:] = np.array(path_dict['UTCI']['stack'])

    # dst.setncattr('crs', CRS.from_epsg(4326),)
    # dst.setncattr('transform', path_dict['UTCI']['tif_meta']['transform'])
    dst.setncattr('nodata', path_dict['UTCI']['tif_meta']['nodata'])
    # dst.setncattr('title', 'check_data')
    if path_dict['UTCI']['upper_limit'] is not None:
        dst.setncattr('upper_limit', path_dict['UTCI']['upper_limit'])
        dst.setncattr('lower_limit', path_dict['UTCI']['lower_limit'])
    dst.setncattr('dates', path_dict['UTCI']['dates'])

    dst.setncattr('history', 'Created ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    min_lat = lats.min()
    max_lat = lats.max()
    min_lon = lons.min()
    max_lon = lons.max()

    startdate = datetime.strptime(path_dict['UTCI']['dates'][0], '%Y-%m-%d')
    startdate = startdate.strftime('%Y-%m-%d %H:%M:%S') 

    global_attrs ={'title': 'Chech data',
                    'Conventions': 'ACDD-1.3, CF-1.11',
                    'conventionsURL': 'http://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html',
                    'keywords': 'DWD, ' + 'UTCI',
                    'keywords_vocabulary': 'GCMD Science Keywords',
                    'cdm_data_type': 'Grid',
                    # 'creator_name': org[dwd_param].institution,
                    # 'creator_email': org[dwd_param].contact,
                    # 'creator_url': 'www.dwd.de',
                    'publisher_name': 'Zentrum für Agrarlandschaftsforschung (ZALF) e.V.',
                    'publisher_email': 'colja.krugmann@zalf.de',
                    'publisher_url': 'www.zalf.de',
                    'date_metadata_modified': datetime.now().strftime('%Y-%m-%d'),
                    'geospatial_bounds': 'POLYGON (({} {}, {} {}, {} {}, {} {}))'.format(
                        min_lon, min_lat, max_lon, min_lat, max_lon, max_lat, min_lon, max_lat
                    ),
                    'geospatial_bounds_crs': 'EPSG:4326',
                    'geospatial_lat_min': str(min_lat),
                    'geospatial_lat_max': str(max_lat),
                    'geospatial_lon_min': str(min_lon),
                    'geospatial_lon_max': str(max_lon),
                    'time_coverage_start': startdate + 'A', # Timezone UTC+100
                    # 'time_coverage_end': str(org['time'][:].max()) + 'A', # Timezone UTC+100, A. 
                    'time_coverage_resolution': 'P1D',
    }

    dst.setncatts(global_attrs)

    # dst.close()
  

In [124]:
nc = Dataset('./netcdfs/test17.nc', 'r', format='NETCDF4')

OSError: [Errno -51] NetCDF: Unknown file format: './netcdfs/test17.nc'

In [120]:
import requests
import xmltodict

url = "http://127.0.0.1:8080/thredds/ncml/data/DWD_Data/zalf_hurs_amber_2011_v1-0_cf_v6.nc"
nc_dict = {}
ncml = requests.get(url)

In [121]:
ncml.text

'<!doctype html><html lang="en"><head><title>HTTP Status 404 – Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 – Not Found</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Message</b> The requested resource [&#47;thredds&#47;ncml&#47;data&#47;DWD_Data&#47;zalf_hurs_amber_2011_v1-0_cf_v6.nc] is not available</p><p><b>Description</b> The origin server did not find a current representation for the target resource or is not willing to disclose that one exists.</p><hr class="line" /><h3>Apache Tomcat/9.0.73</h3></body></html>'

In [118]:
ncml_data = xmltodict.parse(ncml.text)
ncml_data

ExpatError: syntax error: line 1, column 0

In [125]:
conda install -c conda-forge nco


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/colja/miniconda3/envs/.menv

  added / updated specs:
    - nco


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2024.6.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    esmf-8.6.1                 | nompi_h0a5817f_2        23.0 MB  conda-forge
    gsl-2.7                    |       he838d99_0         3.2 MB  conda-forge
    libblas-3.9.0              |16_linux64_openblas          13 KB  conda-forge
    libcblas-3.9.0             |16_linux64_openblas          13 KB  conda-forge
    liblapack-3.9.0            |16_linux64_openblas          13 KB  conda-forge
    libudunits2-2.2.28         |       h40f5838_3          80 KB  conda-forge
    nco-5.2.6                  |       hc167251_0         1.8 MB  conda-forge
    netcdf-fortran-4.6.1   

In [14]:

# Open the GeoTIFF file using rasterio
with rasterio.open('./ftp_download/AWD_0-40cm_2024-06-14.tif') as dataset:
    # Read metadata
    meta = dataset.meta
    
    # Read the affine transformation
    transform = dataset.transform
    
    # Read the coordinate reference system
    crs = dataset.crs
    
    # Read the data
    data = dataset.read()  # Reads all bands

    # Read a specific band (for example, the first band)
    band1 = dataset.read(1)
    
    # Get nodata value
    nodata = dataset.nodata
    
    # Get the width and height of the dataset
    width = dataset.width
    height = dataset.height
    
    # Get the number of bands
    count = dataset.count
    
    # Read the bounding box
    bounds = dataset.bounds

# Store everything in a dictionary
geotiff_info = {
    'meta': meta,
    'transform': transform,
    'crs': crs,
    'data': data,
    'band1': band1,
    'nodata': nodata,
    'width': width,
    'height': height,
    'count': count,
    'bounds': bounds,
}

# Now `geotiff_info` contains all the information about the GeoTIFF file
print(geotiff_info)




{'meta': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 259, 'height': 191, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.1, 0.0, 4.650000000000006,
       0.0, -0.1, 55.050000000000004)}, 'transform': Affine(0.1, 0.0, 4.650000000000006,
       0.0, -0.1, 55.050000000000004), 'crs': CRS.from_epsg(4326), 'data': array([[[-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          1.17666445e+01,  8.86395264e+00,  6.19990253e+00],
        [-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          4.17536259e+00,  2.97724628e+00, -2.62212873e+00],
        [-3.40282347e+38, -3.40282347e+38, -3.40282347e+38, ...,
          2.37905431e+00, -4.16109848e+00, -3.93445301e+00],
        ...,
        [-3.40282347e+38, -2.22427845e+00, -5.01790047e+00, ...,
         -1.19066644e+00, -1.19066644e+00, -1.19066644e+00],
        [-3.40282347e+38, -1.79855525e+00, -2.24067092e+00, ...,
         -3.40282347e+38, -3.40282347e+38, -3.402823

In [59]:
rows, cols = np.indices((height, width))

# Apply the affine transform to convert pixel coordinates to geographic coordinates
x_coords, y_coords = rasterio.transform.xy(transform, rows.flatten(), cols.flatten())
x_coords = np.array(x_coords)
y_coords = np.array(y_coords)

# # Reshape coordinates to match the raster shape
x_coords = x_coords.reshape((height, width))
y_coords = y_coords.reshape((height, width))

y_coords
lats = y_coords[:, 0]
lons = x_coords[0, :]

In [63]:
y_coords
lats = y_coords[:, 0]
lons = x_coords[0, :]

In [66]:
len(lats)

191

In [3]:
a = Affine(0.1, 0.0, 4.650000000000006,
          0.0, -0.1, 55.050000000000004)

In [6]:
a.identity()

Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)

In [36]:
nd.transform 

'4.650000000000006, 0.1, 0.0, 55.050000000000004, 0.0, -0.1'

In [32]:
nd.transform = '0.1, 0.0, 4.650000000000006, 0.0, -0.1, 55.050000000000004'

In [33]:
nd.close()
